
<p style="font-size:21px; font-family:'Times New Roman';">
Read report dates data 
</p>

In [1]:
import pandas as pd
import numpy as np
csv_file_path = '/Users/balmeru/Downloads/QQQQ.csv'
df = pd.read_csv(csv_file_path)


In [2]:
df['market_cap'] = df['prccq'] / df['ajexq'] * df['cshoq']


In [3]:
print(df[['prccq', 'ajexq', 'cshoq', 'market_cap']].head())


   prccq  ajexq  cshoq  market_cap
0  2.875    1.0  2.092     6.01450
1  4.000    1.0  2.094     8.37600
2  4.375    1.0  2.098     9.17875
3  6.750    1.0  2.096    14.14800
4  7.250    1.0  2.163    15.68175


In [4]:
# Calculate upper and lower bounds
df['lower_bound'] = 0.8 * df['market_cap']
df['upper_bound'] = 1.2 * df['market_cap']

In [5]:
df['quarterly_report'] = (df['fqtr'].isin([1, 2, 3])).astype(int)
df['annual_report'] = (df['fqtr'] == 4).astype(int)


In [6]:
df['fyr'] = pd.to_numeric(df['fyr'], errors='coerce')
df = df.sort_values(by=['tic', 'fyearq', 'fqtr'])
ticker_change = df['tic'] != df['tic'].shift(1)
df['fyr_change_dummy'] = ((df['fyr'] != df['fyr'].shift(1)) & (~ticker_change)).astype(int)

# Fill the first occurrence of each ticker group with 0
df.loc[df.groupby('tic').head(1).index, 'fyr_change_dummy'] = 0


In [7]:
df['rdq'] = pd.to_datetime(df['rdq'], errors='coerce')
df['fyearq'] = pd.to_numeric(df['fyearq'], errors='coerce')
df['fqtr'] = pd.to_numeric(df['fqtr'], errors='coerce')

grouped = df.groupby('tic', group_keys=False)

# Function to calculate consecutive differences
def calculate_diff(group):
    # Sort the data within each group by the fiscal year and quarter in ascending order
    group = group.sort_values(['fyearq', 'fqtr'])
    
    # Create a shifted column for the previous rdq
    group['prev_rdq'] = group['rdq'].shift(1)
    group['prev_fqtr'] = group['fqtr'].shift(1)
    group['prev_fyearq'] = group['fyearq'].shift(1)
    
    # Calculate the differences
    group['diff'] = group['rdq'] - group['prev_rdq']
    
    # Remove differences where quarters and years are not consecutive
    mask = (group['fqtr'] == 1) & (group['prev_fqtr'] == 4) & (group['fyearq'] == group['prev_fyearq'] + 1)
    mask |= (group['fqtr'] > 1) & (group['fqtr'] == group['prev_fqtr'] + 1) & (group['fyearq'] == group['prev_fyearq'])
    
    group['diff'] = group['diff'].where(mask)
    
    # Drop helper columns
    group = group.drop(columns=['prev_rdq', 'prev_fqtr', 'prev_fyearq'])
    
    return group

# Apply the function to each ticker group directly in the original DataFrame
df = grouped.apply(calculate_diff)
df.loc[df['fyr_change_dummy'] == 1, 'diff'] = pd.NaT
df = df.reset_index(drop=True)

/var/folders/ds/rfyxvvb945vdt8znlkx31w1m00cj2y/T/ipykernel_20564/1492790700.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = grouped.apply(calculate_diff)


#### Let's split the distance for quarterly and annual reports

In [8]:
df['quarter_distance'] = df['diff'].where(df['quarterly_report'] == 1, pd.NaT)
df['annual_distance'] = df['diff'].where(df['annual_report'] == 1, pd.NaT)
print(df[['tic', 'rdq', 'fyr', 'fyearq', 'fqtr', 'fyr_change_dummy', 'annual_distance', 'quarter_distance', 'diff']].head())


     tic        rdq  fyr  fyearq  fqtr  fyr_change_dummy annual_distance  \
0  0015B        NaT   10    1983     3                 0             NaT   
1  0015B 1984-01-16   10    1983     4                 0             NaT   
2  0015B 1984-03-15   10    1984     1                 0             NaT   
3  0015B 1984-05-25   10    1984     2                 0             NaT   
4  0015B 1984-08-27   10    1984     3                 0             NaT   

  quarter_distance    diff  
0              NaT     NaT  
1              NaT     NaT  
2          59 days 59 days  
3          71 days 71 days  
4          94 days 94 days  


In [9]:
# Initialize above_cshoq, below_cshoq, above_datafqtr, and below_datafqtr as NaN
df['above_cshoq'] = np.nan
df['below_cshoq'] = np.nan
df['above_datafqtr'] = np.nan
df['below_datafqtr'] = np.nan

# Find rows where fyr_change_dummy is 1 and cshoq is NaN
missing_cshoq_indices = df[(df['fyr_change_dummy'] == 1) & df['cshoq'].isna()].index

# Iterate over the missing cshoq indices to find above_cshoq, below_cshoq, above_datafqtr, and below_datafqtr
for index in missing_cshoq_indices:
    tic = df.at[index, 'tic']
    current_datafqtr = df.at[index, 'datafqtr']
    
    # Find the row above with a non-missing cshoq value
    above_rows = df[(df['tic'] == tic) & (df['datafqtr'] < current_datafqtr)].sort_values(by='datafqtr', ascending=False)
    for _, above_row in above_rows.iterrows():
        if not pd.isna(above_row['cshoq']):
            df.at[index, 'above_cshoq'] = above_row['cshoq']
            df.at[index, 'above_datafqtr'] = above_row['datafqtr']
            break
    
    # Find the row below with a non-missing cshoq value
    below_rows = df[(df['tic'] == tic) & (df['datafqtr'] > current_datafqtr)].sort_values(by='datafqtr', ascending=True)
    for _, below_row in below_rows.iterrows():
        if not pd.isna(below_row['cshoq']):
            df.at[index, 'below_cshoq'] = below_row['cshoq']
            df.at[index, 'below_datafqtr'] = below_row['datafqtr']
            break

# Print rows where fyr_change_dummy is 1 and cshoq is NaN
print(df[(df['fyr_change_dummy'] == 1) & df['cshoq'].isna()])

/var/folders/ds/rfyxvvb945vdt8znlkx31w1m00cj2y/T/ipykernel_20564/4092613696.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1988Q4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'above_datafqtr'] = above_row['datafqtr']
/var/folders/ds/rfyxvvb945vdt8znlkx31w1m00cj2y/T/ipykernel_20564/4092613696.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1989Q4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'below_datafqtr'] = below_row['datafqtr']


          GVKEY    datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
22         2484  1988-09-30    1989     1    6   INDL      C      D     STD   
1078      23929  1996-12-31    1997     1    9   INDL      C      D     STD   
2993       6726  1979-12-31    1980     1    9   INDL      C      D     STD   
3289       3688  1980-09-30    1981     1    6   INDL      C      D     STD   
3821      14287  1990-03-31    1990     1   12   INDL      C      D     STD   
...         ...         ...     ...   ...  ...    ...    ...    ...     ...   
1072221   18484  1994-11-30    1995     1    8   INDL      C      D     STD   
1075104   24318  1999-03-31    1999     1   12   INDL      C      D     STD   
1075351  100598  2000-03-31    2000     1   12   INDL      C      D     STD   
1076701   30751  1997-03-31    1997     1   12   INDL      C      D     STD   
1080188   11656  1991-05-31    1991     1    2   INDL      C      D     STD   

            tic  ...  quarterly_report annual_repor

In [10]:
df['diff_fqtr'] = np.nan

# Calculate diff_fqtr for rows where both below_datafqtr and above_datafqtr are given
mask = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['below_datafqtr'].notna() & df['above_datafqtr'].notna()

for index, row in df[mask].iterrows():
    # Extract year and quarter from below_datafqtr and above_datafqtr
    below_year, below_quarter = row['below_datafqtr'].split('Q')
    above_year, above_quarter = row['above_datafqtr'].split('Q')

    # Convert year and quarter to integers
    below_year = int(below_year)
    below_quarter = int(below_quarter)
    above_year = int(above_year)
    above_quarter = int(above_quarter)

    # Calculate the difference in quarters
    diff_fqtr = (below_year - above_year) * 4 + (below_quarter - above_quarter)

    # Update the diff_fqtr column with the calculated difference
    df.at[index, 'diff_fqtr'] = diff_fqtr


In [11]:
df['diff_cshoq'] = df.apply(lambda row: row['below_cshoq'] - row['above_cshoq'] if pd.notna(row['above_cshoq']) and pd.notna(row['below_cshoq']) else np.nan, axis=1)



In [12]:
df['new_cshoq'] = np.nan

# Calculate new_cshoq for rows where fyr_change_dummy is 1, cshoq is NaN, and diff_cshoq and diff_fqtr are not NaN
mask = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['diff_cshoq'].notna() & df['diff_fqtr'].notna()

for index, row in df[mask].iterrows():
    # Calculate the difference between datafqtr and above_datafqtr
    above_year, above_quarter = row['above_datafqtr'].split('Q')
    data_year, data_quarter = row['datafqtr'].split('Q')
    
    above_year = int(above_year)
    above_quarter = int(above_quarter)
    data_year = int(data_year)
    data_quarter = int(data_quarter)
    
    difference = (data_year - above_year) * 4 + (data_quarter - above_quarter)
    
    # Calculate new_cshoq value
    new_cshoq = row['above_cshoq'] + (row['diff_cshoq'] / row['diff_fqtr']) * difference
    
    # Update the new_cshoq column with the calculated value
    df.at[index, 'new_cshoq'] = new_cshoq

print(df[['tic', 'fyr', 'cshoq', 'above_cshoq', 'new_cshoq']][mask])


            tic  fyr  cshoq  above_cshoq   new_cshoq
22        0015B    6    NaN       11.762   11.773500
1078      0147A    9    NaN        6.548    6.551500
2993      2788A    9    NaN        8.436    8.447000
3289      2919B    6    NaN        3.126    3.126000
3821      3213B   12    NaN       71.085   71.150000
...         ...  ...    ...          ...         ...
1071759    WWON   12    NaN       16.330   21.196333
1072221  WWWW.1    8    NaN        8.804   36.343333
1075104    XL.1   12    NaN      111.804  117.062667
1076701   XPRSA   12    NaN       12.086   12.101500
1080188    YORK    2    NaN        8.477    8.700667

[371 rows x 5 columns]


In [13]:
# Update new_cshoq for rows where fyr_change_dummy is 1, cshoq is NaN, and above_cshoq or below_cshoq is NaN
mask_above_nan = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['above_cshoq'].isna()
mask_below_nan = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['below_cshoq'].isna()

# Set new_cshoq equal to below_cshoq when above_cshoq is NaN
df.loc[mask_above_nan, 'new_cshoq'] = df.loc[mask_above_nan, 'below_cshoq']

# Set new_cshoq equal to above_cshoq when below_cshoq is NaN
df.loc[mask_below_nan, 'new_cshoq'] = df.loc[mask_below_nan, 'above_cshoq']

print(df[['tic', 'fyr', 'cshoq', 'above_cshoq', 'below_cshoq', 'new_cshoq']][mask_above_nan | mask_below_nan])


            tic  fyr  cshoq  above_cshoq  below_cshoq  new_cshoq
4765      3ACDX   12    NaN        2.100          NaN      2.100
37397    3STPHF    9    NaN        3.969          NaN      3.969
51322     5841A    9    NaN       85.800          NaN     85.800
51545     5853B    6    NaN        2.669          NaN      2.669
53821     6103B   12    NaN        3.416          NaN      3.416
55474     6733B    1    NaN        8.987          NaN      8.987
57620     7556A    6    NaN       16.111          NaN     16.111
58716     8550B   12    NaN       56.286          NaN     56.286
60530     9901B    8    NaN       15.764          NaN     15.764
237388    CGA.1   12    NaN          NaN      312.110    312.110
238832     CGNE   12    NaN       60.443          NaN     60.443
239820    CHAM.    3    NaN          NaN          NaN        NaN
390661      FA1    6    NaN       25.075          NaN     25.075
485586      HFO    9    NaN        0.977          NaN      0.977
590894    LCNAF    9    N

In [14]:
# Replace original cshoq values with new_cshoq for rows where fyr_change_dummy is 1 and cshoq is NaN
mask_replace_cshoq = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna()

# Update cshoq with new_cshoq values
df.loc[mask_replace_cshoq, 'cshoq'] = df.loc[mask_replace_cshoq, 'new_cshoq']

print(df[['tic', 'fyr', 'cshoq']][mask_replace_cshoq])


            tic  fyr       cshoq
22        0015B    6   11.773500
1078      0147A    9    6.551500
2993      2788A    9    8.447000
3289      2919B    6    3.126000
3821      3213B   12   71.150000
...         ...  ...         ...
1072221  WWWW.1    8   36.343333
1075104    XL.1   12  117.062667
1075351     XLL   12  104.860000
1076701   XPRSA   12   12.101500
1080188    YORK    2    8.700667

[393 rows x 3 columns]


In [15]:
df['market_cap'] = df['prccq'] / df['ajexq'] * df['cshoq']

In [16]:
# Calculate upper and lower bounds
df['lower_bound'] = 0.8 * df['market_cap']
df['upper_bound'] = 1.2 * df['market_cap']

In [17]:
df['median_distance'] = np.nan

# Ensure the 'median_distance' column is updated for fiscal year change rows
df['median_distance'] = np.where(df['fyr_change_dummy'] == 0, np.nan, df['median_distance'])

# Initialize variables to count instances with all empty and non-empty distances
num_instances_all_empty = 0
num_instances_non_empty = 0

# Initialize sets to store tickers with non-empty, empty, and mixed distances
non_empty_distances_tickers = set()
empty_distances_tickers = set()
mixed_distances_tickers = set()

# Filter rows where fiscal year change occurred
fyr_change_rows = df[df['fyr_change_dummy'] == 1]

# Iterate over rows where fiscal year change occurred
for index, fyr_change_row in fyr_change_rows.iterrows():
    # Determine if it's a quarterly or annual report
    is_quarterly_report = fyr_change_row['quarterly_report'] == 1

    # Filter similar market cap tickers excluding own ticker
    similar_cap_tickers = df[(df['market_cap'] >= fyr_change_row['lower_bound']) &
                             (df['market_cap'] <= fyr_change_row['upper_bound']) &
                             (df['tic'] != fyr_change_row['tic'])]

    # Determine relevant distance column and filter criteria
    distance_col = 'quarter_distance' if is_quarterly_report else 'annual_distance'
    year_col = 'fyearq'
    years = [fyr_change_row['fyearq'], fyr_change_row['fyearq'] + 1, fyr_change_row['fyearq'] - 1]

    # Extract distances
    distances = similar_cap_tickers[similar_cap_tickers[year_col].isin(years)][distance_col].dropna().apply(lambda x: pd.Timedelta(x).days)

    # Check if distances list is empty
    if distances.empty:
        num_instances_all_empty += 1
        if fyr_change_row['tic'] in non_empty_distances_tickers:
            mixed_distances_tickers.add(fyr_change_row['tic'])
        else:
            empty_distances_tickers.add(fyr_change_row['tic'])
    else:
        median_distance = distances.median()
        df.loc[index, 'median_distance'] = median_distance
        num_instances_non_empty += 1
        if fyr_change_row['tic'] in empty_distances_tickers:
            mixed_distances_tickers.add(fyr_change_row['tic'])
        else:
            non_empty_distances_tickers.add(fyr_change_row['tic'])

# Calculate counts for unique tickers
num_non_empty_distances_tickers = len(non_empty_distances_tickers - mixed_distances_tickers)
num_empty_distances_tickers = len(empty_distances_tickers - mixed_distances_tickers)
num_mixed_distances_tickers = len(mixed_distances_tickers)
num_unique_fyr_change_tickers = fyr_change_rows['tic'].nunique()

# Display the results
print(f"Number of instances with all empty distances: {num_instances_all_empty}")
print(f"Number of instances with non-empty distances: {num_instances_non_empty}")
print(f"Number of unique tickers with non-empty distances: {num_non_empty_distances_tickers}")
print(f"Number of unique tickers with all empty distances: {num_empty_distances_tickers}")
print(f"Number of unique tickers with mixed distances: {num_mixed_distances_tickers}")
print(f"Total number of unique tickers with fiscal year changes: {num_unique_fyr_change_tickers}")


Number of instances with all empty distances: 31
Number of instances with non-empty distances: 2157
Number of unique tickers with non-empty distances: 1947
Number of unique tickers with all empty distances: 23
Number of unique tickers with mixed distances: 8
Total number of unique tickers with fiscal year changes: 1978


In [18]:
summary_stats = df['median_distance'].describe()
formatted_stats = summary_stats.apply(lambda x: f'{x:.2f}')

print(formatted_stats)

count    2157.00
mean       90.55
std         4.29
min        68.00
25%        90.00
50%        90.00
75%        91.00
max       132.00
Name: median_distance, dtype: object


In [19]:
# Filter rows where fiscal year change occurred
fyr_change_rows = df[df['fyr_change_dummy'] == 1]

# Iterate over rows where fiscal year change occurred
for index, fyr_change_row in fyr_change_rows.iterrows():
    # Check if fiscal year change happened in quarterly or annual report
    is_quarterly_report = fyr_change_row['quarterly_report'] == 1

    # Determine which column to assign median distance
    median_distance_column = 'annual_median' if not is_quarterly_report else 'quarterly_median'

    # Update the median_distance column accordingly
    df.loc[index, median_distance_column] = fyr_change_row['median_distance']

# Set NaN for rows where fyr_change_dummy is 0
df.loc[df['fyr_change_dummy'] == 0, ['annual_median', 'quarterly_median']] = np.nan


In [20]:
annual_median_summary = df['annual_median'].describe()
print("Summary statistics for Annual Median:")
print(annual_median_summary)

quarterly_median_summary = df['quarterly_median'].describe()
print("\nSummary statistixcs for Quarterly Median:")
print(quarterly_median_summary)


Summary statistics for Annual Median:
count     55.000000
mean     115.045455
std        6.437213
min      101.000000
25%      112.000000
50%      116.000000
75%      119.000000
max      132.000000
Name: annual_median, dtype: float64

Summary statistixcs for Quarterly Median:
count    2102.000000
mean       89.909134
std         1.291728
min        68.000000
25%        89.500000
50%        90.000000
75%        91.000000
max        96.000000
Name: quarterly_median, dtype: float64


In [21]:
df = pd.DataFrame(df)
df.to_csv('/Users/balmeru/Downloads/AI.csv', index=False)

#print("DataFrame saved to 'AI.csv'")
